In [ ]:
import pandas as pd
import sys
sys.path.append("..")
import helper
import re
import html
import matplotlib.pyplot as plt
from tqdm import tqdm
import swifter
import splitwords.core as sc
from pyvi import ViTokenizer, ViPosTagger
from pyvi import ViUtils

c:\Users\tungq\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../data/clean_comments.csv')
df.head()

,emotion,video_id,author,comment
0,vui vẻ,_pMLRYGx9Is,@ottux2707,ai đó rì viu dhhp cho em vớiii
1,vui vẻ,_pMLRYGx9Is,@minhchau3090,tính cả mình thì khóa mình có 5 người trong bộ...
2,vui vẻ,_pMLRYGx9Is,@yennhile8168,ê em là 2k8 lo xa đag tìm trường đhoc e xem vi...
3,vui vẻ,_pMLRYGx9Is,@tuantruong2080,anh làm p2 cho 2k7 đc ko
4,vui vẻ,_pMLRYGx9Is,@johnnyphim5572,ftu giỏi lắm đó


In [47]:
def abbreviations_fix(text: str):
    """
    Fixes abbreviations in the text.
    """
    global count
    added_accent_text = ""
    try:
        added_accent_text = ViUtils.add_accents(text)
    except:
        added_accent_text = text
    finally:
        change_state = False
        text = added_accent_text.lower()
        words = text.split()
        for i in range(len(words)):
            if words[i] in abbr_dict:
                words[i] = abbr_dict[words[i]]
                change_state = True
                count += 1
        text = ' '.join(words)
        return text

def get_abbreviation_dict(file: str = './splitwords/dicts/teencode_raw.bak'):
    """
    Reads the abbreviation dictionary from a file.
    """
    with open(file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    abbr_dict = {}
    for line in lines:
        if line.strip() == '':
            continue
        parts = line.strip().split('\t')
        if len(parts) != 2:
            continue
        abbr, full = parts
        abbr_dict[abbr] = full
    return abbr_dict

print("Loading abbreviation dictionary...")
abbr_dict = get_abbreviation_dict()
abbr_dict

Loading abbreviation dictionary...


{'ctrai': 'con trai',
 'khôg': 'không',
 'bme': 'bố mẹ',
 'cta': 'chúng ta',
 'mih': 'mình',
 'mqh': 'mối quan hệ',
 'cgai': 'con gái',
 'nhữg': 'những',
 'mng': 'mọi người',
 'svtn': 'sinh viên tình nguyện',
 'r': 'rồi',
 'qtam': 'quan tâm',
 'thươg': 'thương',
 'qtâm': 'quan tâm',
 'chug': 'chung',
 'trườg': 'trường',
 'thoy': 'thôi',
 'đki': 'đăng ký',
 'atsm': 'ảo tưởng sức mạnh',
 'ạk': 'ạ',
 'cv': 'công việc',
 'vch': 'vãi chưởng',
 'cùg': 'cùng',
 'pn': 'bạn',
 'pjt': 'biết',
 'thjk': 'thích',
 'keke': 'ce ce',
 'ktra': 'kiểm tra',
 'nek': 'nè',
 'cgái': 'con gái',
 'nthe': 'như thế',
 'chúg': 'chúng',
 'kái': 'cái',
 'tìh': 'tình',
 'phòg': 'phòng',
 'lòg': 'lòng',
 'từg': 'từng',
 'rằg': 'rằng',
 'sốg': 'sống',
 'thuj': 'thôi',
 'thuơng': 'thương',
 'càg': 'càng',
 'đky': 'đăng ký',
 'bằg': 'bằng',
 'sviên': 'sinh viên',
 'ák': 'á',
 'đág': 'đáng',
 'nvay': 'như vậy',
 'nhjeu': 'nhiều',
 'xg': 'xuống',
 'zồi': 'rồi',
 'trag': 'trang',
 'zữ': 'dữ',
 'atrai': 'anh trai',
 'kte':

In [48]:
tqdm.pandas()
count = 0
df['clean_comment'] = df['comment'].progress_apply(abbreviations_fix)
print(f"Number of abbreviations fixed: {count}")

100%|██████████| 5413/5413 [00:07<00:00, 733.10it/s] 

Number of abbreviations fixed: 2585


In [40]:
df.head(50)

,emotion,video_id,author,comment,clean_comment
0,vui vẻ,_pMLRYGx9Is,@ottux2707,ai đó rì viu dhhp cho em vớiii,ăi đó gì viu dhhp cho em vớiii
1,vui vẻ,_pMLRYGx9Is,@minhchau3090,tính cả mình thì khóa mình có 5 người trong bộ...,tính cả mình thì khóa mình có 5 người trong bộ...
2,vui vẻ,_pMLRYGx9Is,@yennhile8168,ê em là 2k8 lo xa đag tìm trường đhoc e xem vi...,ê em là 2k8 lô xã đang tìm trường đại học em x...
3,vui vẻ,_pMLRYGx9Is,@tuantruong2080,anh làm p2 cho 2k7 đc ko,anh làm p2 cho 2k7 được không
4,vui vẻ,_pMLRYGx9Is,@johnnyphim5572,ftu giỏi lắm đó,ftu giỏi lắm đó
5,vui vẻ,_pMLRYGx9Is,@TCon-kt9hb,ơ kìa học viện hoàng gia đâu rồi anh vui vẻ th...,ơ kìa học viện hoàng gia đâu rồi anh vui vẻ th...
6,vui vẻ,_pMLRYGx9Is,@manhha9227,ông này lấy ảnh đại học hùng vương ở phú thọ n...,ông nàỹ lấy ảnh đại học hùng vương ở phú thọ n...
7,vui vẻ,_pMLRYGx9Is,@C--NgoGiaHan,hong có trường văn hoá hả,hồng có trường văn hòá hả
8,vui vẻ,_pMLRYGx9Is,@PhamTheAnh.,4 20 thật ra là to nhất đông nam á,4 20 thật ra là tổ nhất đông nam á
9,vui vẻ,_pMLRYGx9Is,@ThuanNguyen-hx6pm,t vẫn cay môn đồ họa kĩ thuật má shibaaaaa,t vẫn cây môn đồ họa kĩ thuật má shibaaaaa


In [49]:
# Remove col comment
df = df.drop(columns=['comment'])
# Rename col clean_comment to comment
df = df.rename(columns={'clean_comment': 'comment'})
# Save to csv
df.to_csv('../data/accend_added.csv', index=False)